# Data preparation - creating dataset for modeling
Code in this file is used to load raw data (PM data extracted using *data_collection_air_pollution.ipynb* and atmospheric data from IMGW archive - link and more info in README)

In [31]:
import pandas as pd
import holidays
import numpy as np

## Atmospheric data
### Data import

In [19]:
# Full year data - already filtered for Gdańsk
atm_df2021 = pd.read_csv("atm data/s_d_t_155_2021.csv", sep=",", encoding="latin1", header=None)
atm_df2022 = pd.read_csv("atm data/s_d_t_155_2022.csv", sep=",", encoding="latin1", header=None)
atm_df2023 = pd.read_csv("atm data/s_d_t_155_2023.csv", sep=",", encoding="latin1", header=None)
atm_df2024 = pd.read_csv("atm data/s_d_t_155_2024.csv", sep=",", encoding="latin1", header=None)

### Data operations - atmospheric conditions

In [20]:
atm_df = pd.concat([atm_df2021, atm_df2022, atm_df2023, atm_df2024], ignore_index=True)
atm_df = atm_df[[2, 3, 4, # year, month, day
                 7, # wind speed
                 9, # temperature
                 13, # humidity
                 15, # pressure
                 19, # precip day
                 21 # precip night
                 ]]
atm_df["Prec"] = atm_df[19] + atm_df[21]
atm_df["Date"] = pd.to_datetime(atm_df[[2, 3, 4]].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')
atm_df

,2,3,4,7,9,13,15,19,21,Prec,Date
0,2021,1,1,4.8,-1.9,97.4,1008.9,0.0,0.4,0.4,2021-01-01
1,2021,1,2,4.1,-0.4,96.9,1014.4,0.0,0.0,0.0,2021-01-02
2,2021,1,3,4.4,-0.1,94.8,1019.7,0.0,2.3,2.3,2021-01-03
3,2021,1,4,5.3,0.7,92.1,1019.5,0.0,0.4,0.4,2021-01-04
4,2021,1,5,6.1,1.8,93.5,1016.2,7.9,7.7,15.6,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...
1272,2024,6,26,3.0,21.9,72.1,1016.0,0.0,0.0,0.0,2024-06-26
1273,2024,6,27,3.0,24.7,61.4,1011.7,0.0,0.0,0.0,2024-06-27
1274,2024,6,28,3.5,23.2,75.1,1010.5,7.9,0.1,8.0,2024-06-28
1275,2024,6,29,3.3,18.7,74.1,1016.8,0.0,0.0,0.0,2024-06-29


## PM data
### Data import

In [21]:
pm_2021 = pd.read_csv("pomiar2021.csv")
pm_2022 = pd.read_csv("pomiar2022.csv")
pm_2023 = pd.read_csv("pomiar2023.csv")
pm_2023_1 = pd.read_csv("pomiar2023_1.csv")
pm_2024 = pd.read_csv("pomiar2024.csv")

Due to error with data extraction, 2023 data has to be fixed. File pm_2023 contains data for PM2.5 (full year) and PM10(missing last few months). FIle pm_2023_1 contains data for the missing months

In [22]:
pm_2023 = pd.merge(pm_2023,
                   pm_2023_1,
                   how="left",
                   on="date")
pm_2023["PM2.5"] = pm_2023["PM2.5_x"]
pm_2023["PM10"] = pm_2023["PM10_x"].fillna(pm_2023["PM10_y"])
pm_2023 = pm_2023.drop(columns=["PM2.5_x", "PM2.5_y", "PM10_x", "PM10_y"])
pm_2023

,date,PM2.5,PM10
0,2023-01-01,8.181665,9.926444
1,2023-01-02,15.713404,18.891842
2,2023-01-03,3.289819,3.553683
3,2023-01-04,11.146108,12.550330
4,2023-01-05,2.650624,2.717800
...,...,...,...
359,2023-12-27,4.838207,5.487628
360,2023-12-28,14.481641,17.289795
361,2023-12-29,5.445825,5.871910
362,2023-12-30,3.899353,4.007467


In [23]:
pm = pd.concat([pm_2021, pm_2022, pm_2023, pm_2024], ignore_index=True)
pm["date"] = pd.to_datetime(pm["date"], format='%Y-%m-%d')

## Creating dataset
Code below contains a process of adding the features that will be used in model (more features will be added after preparing the dataset manually in Excel):
- IsWeekend - 1 if the day is Saturday or Sunday
- IsHoliday - 1 if the day is public holiday in Poland
- Risk and FutureRisk - the most important features - they will be used as labels

In [ ]:
df = pd.merge(atm_df, pm, how="left", left_on="Date", right_on="date")
df = df[df["PM2.5"].notna()]
df = df[df["PM10"].notna()]
df.drop(columns=["date", 4, 19, 21], inplace=True)
df.columns = ["Year", "Month", "WindSpeed", "Temperature", "Humidity", "Pressure", "Precipitation", "Date", "PM2.5", "PM10"]
pl_holidays = holidays.Poland(years=range(2021, 2025)) # Create a list of Polish holidays from 2015 to 2024
df["IsWeekend"] = df["Date"].dt.weekday > 4  # Weekday is 0-4 (Monday to Friday), weekend is 5-6 (Saturday and Sunday)
df["IsHoliday"] = df["Date"].isin(pl_holidays) # Check if the date is a holiday
# Convert boolean columns to integers (0 and 1)
df["IsWeekend"] = df["IsWeekend"].astype(int)
df["IsHoliday"] = df["IsHoliday"].astype(int)

/tmp/ipykernel_48064/4102719230.py:8: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['IsHoliday'] = df['Date'].isin(pl_holidays) # Check if the date is a holiday


In [ ]:
df["Risk"] = np.where((df["PM10"] >= 50) | (df["PM2.5"] >= 25), 1, 0)
df["FutureRisk"] = df["Risk"].shift(-1).fillna(0).astype(int) # Shift the risk column to create a future risk column
df.drop(["Risk"], inplace=True, axis=1) # Drop the original risk column

In [36]:
df.to_excel('data.xlsx') # Save the DataFrame to an Excel file